In [ ]:

%run -m spacy download en_core_web_sm

In [38]:
import os
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy
import string
import pandas as pd
punctuations = string.punctuation
from spacy.lang.en import English
parser = English()
from spacy.lang.en.stop_words import STOP_WORDS

from IPython.display import display, HTML
display(HTML(data="""
<style> div#notebook-container { width: 95%; } div#menubar-container { width: 95%; } div#maintoolbar-container { width: 99%; } </style>

"""))
nlp.max_length = 21000000
parser.max_length = 21000000

In [42]:
pl = "OF Mans First Disobedience, and the Fruit Of that Forbidden Tree, whose mortal tast Brought Death into the World, and all our woe, With loss of Eden, till one greater Man Restore us, and regain the blissful Seat, Sing Heav'nly Muse, that on the secret top Of Oreb, or of Sinai, didst inspire That Shepherd, who first taught the chosen Seed,In the Beginning how the Heav'ns and Earth Rose out of Chaos: or if Sion Hill Delight thee more, and Siloa's brook that flow'd Fast by the Oracle of God; I thence Invoke thy aid to my adventrous Song, That with no middle flight intends to soar Above th' Aonian Mount, while it pursues Things unattempted yet in Prose or Rhime."
pl = pl.lower()
doc = nlp(pl)
for token in doc:
    print(token.text, token.pos_)

of ADP
mans NOUN
first ADJ
disobedience NOUN
, PUNCT
and CCONJ
the DET
fruit NOUN
of ADP
that DET
forbidden VERB
tree NOUN
, PUNCT
whose DET
mortal ADJ
tast NOUN
brought VERB
death NOUN
into ADP
the DET
world NOUN
, PUNCT
and CCONJ
all DET
our DET
woe NOUN
, PUNCT
with ADP
loss NOUN
of ADP
eden ADJ
, PUNCT
till ADP
one NUM
greater ADJ
man NOUN
restore VERB
us PRON
, PUNCT
and CCONJ
regain VERB
the DET
blissful ADJ
seat NOUN
, PUNCT
sing VERB
heav'nly ADV
muse NOUN
, PUNCT
that ADP
on ADP
the DET
secret ADJ
top NOUN
of ADP
oreb NOUN
, PUNCT
or CCONJ
of ADP
sinai NOUN
, PUNCT
didst NOUN
inspire NOUN
that DET
shepherd NOUN
, PUNCT
who PRON
first ADV
taught VERB
the DET
chosen VERB
seed NOUN
, PUNCT
in ADP
the DET
beginning NOUN
how ADV
the DET
heav'ns PROPN
and CCONJ
earth NOUN
rose VERB
out ADP
of ADP
chaos NOUN
: PUNCT
or CCONJ
if ADP
sion NOUN
hill NOUN
delight NOUN
thee PRON
more ADJ
, PUNCT
and CCONJ
siloa NOUN
's PART
brook NOUN
that DET
flow'd X
fast ADV
by ADP
the DET
oracle NOUN


In [43]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

of of ADP IN prep xx True True
mans man NOUN NNS pobj xxxx True False
first first ADJ JJ amod xxxx True True
disobedience disobedience NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
the the DET DT det xxx True True
fruit fruit NOUN NN conj xxxx True False
of of ADP IN prep xx True True
that that DET DT det xxxx True True
forbidden forbid VERB VBN amod xxxx True False
tree tree NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
whose whose DET WP$ poss xxxx True True
mortal mortal ADJ JJ amod xxxx True False
tast tast NOUN NN nsubj xxxx True False
brought bring VERB VBD relcl xxxx True False
death death NOUN NN dobj xxxx True False
into into ADP IN prep xxxx True True
the the DET DT det xxx True True
world world NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
all all DET PDT predet xxx True True
our -PRON- DET PRP$ poss xxx True True
woe woe NOUN NN conj xxx True False
, , PUNCT , pu

# Try to print only the nouns

Hint:

Your code could look something like this

``` python

for token in doc:
    if token.pos_ == "    ":
        print(token.text)
```

forbidden VERB
brought VERB
restore VERB
regain VERB
sing VERB
taught VERB
chosen VERB
rose VERB
invoke VERB
intends VERB
soar VERB
pursues VERB


# Tokenization

In [19]:
tokens = [token.text for token in doc]
tokens

['of',
 'mans',
 'first',
 'disobedience',
 ',',
 'and',
 'the',
 'fruit',
 'of',
 'that',
 'forbidden',
 'tree',
 ',',
 'whose',
 'mortal',
 'tast',
 'brought',
 'death',
 'into',
 'the',
 'world',
 ',',
 'and',
 'all',
 'our',
 'woe',
 ',',
 'with',
 'loss',
 'of',
 'eden',
 ',',
 'till',
 'one',
 'greater',
 'man',
 'restore',
 'us',
 ',',
 'and',
 'regain',
 'the',
 'blissful',
 'seat',
 ',',
 'sing',
 "heav'nly",
 'muse',
 ',',
 'that',
 'on',
 'the',
 'secret',
 'top',
 'of',
 'oreb',
 ',',
 'or',
 'of',
 'sinai',
 ',',
 'didst',
 'inspire',
 'that',
 'shepherd',
 ',',
 'who',
 'first',
 'taught',
 'the',
 'chosen',
 'seed',
 ',',
 'in',
 'the',
 'beginning',
 'how',
 'the',
 "heav'ns",
 'and',
 'earth',
 'rose',
 'out',
 'of',
 'chaos',
 ':',
 'or',
 'if',
 'sion',
 'hill',
 'delight',
 'thee',
 'more',
 ',',
 'and',
 'siloa',
 "'s",
 'brook',
 'that',
 "flow'd",
 'fast',
 'by',
 'the',
 'oracle',
 'of',
 'god',
 ';',
 'i',
 'thence',
 'invoke',
 'thy',
 'aid',
 'to',
 'my',
 'a

# Remove Stopwords

In [22]:
[token for token in tokens if token not in STOP_WORDS]

['mans',
 'disobedience',
 ',',
 'fruit',
 'forbidden',
 'tree',
 ',',
 'mortal',
 'tast',
 'brought',
 'death',
 'world',
 ',',
 'woe',
 ',',
 'loss',
 'eden',
 ',',
 'till',
 'greater',
 'man',
 'restore',
 ',',
 'regain',
 'blissful',
 'seat',
 ',',
 'sing',
 "heav'nly",
 'muse',
 ',',
 'secret',
 'oreb',
 ',',
 'sinai',
 ',',
 'didst',
 'inspire',
 'shepherd',
 ',',
 'taught',
 'chosen',
 'seed',
 ',',
 'beginning',
 "heav'ns",
 'earth',
 'rose',
 'chaos',
 ':',
 'sion',
 'hill',
 'delight',
 'thee',
 ',',
 'siloa',
 'brook',
 "flow'd",
 'fast',
 'oracle',
 'god',
 ';',
 'invoke',
 'thy',
 'aid',
 'adventrous',
 'song',
 ',',
 'middle',
 'flight',
 'intends',
 'soar',
 'th',
 "'",
 'aonian',
 'mount',
 ',',
 'pursues',
 'things',
 'unattempted',
 'prose',
 'rhime',
 '.']

# Lemmatazation

In [46]:
filtered_tokens = [token for token in doc if not token.is_stop]
filtered_tokens = [token for token in filtered_tokens if token.text not in punctuations]
for token in filtered_tokens:
    print(token.lemma_)


man
disobedience
fruit
forbid
tree
mortal
tast
bring
death
world
woe
loss
eden
till
great
man
restore
regain
blissful
seat
sing
heav'nly
muse
secret
oreb
sinai
didst
inspire
shepherd
teach
choose
seed
beginning
heav'ns
earth
rise
chaos
sion
hill
delight
thee
siloa
brook
flow'd
fast
oracle
god
invoke
thy
aid
adventrous
song
middle
flight
intend
soar
th
aonian
mount
pursue
thing
unattempted
prose
rhime


# Dependency Parsing

In [48]:
displacy.render(doc, style="dep")

In [47]:
for token in filtered_tokens:
    print(token.text, [child for child in token.children])

mans []
disobedience [first, ,, and, fruit]
fruit [the, of, ,, and, woe]
forbidden []
tree [that, forbidden, ,, brought]
mortal []
tast [whose, mortal]
brought [tast, death, into]
death []
world [the]
woe [all, our, ,, with]
loss [of]
eden []
till []
greater []
man [one, greater]
restore [of, ,, till, man, us, ,, and, regain]
regain [seat, ,, sing, inspire]
blissful []
seat [the, blissful]
sing [muse, ,]
heav'nly []
muse [heav'nly]
secret []
oreb []
sinai []
didst []
inspire [that, on, ,, didst, shepherd, ,, in, rose]
shepherd [that, ,, taught]
taught [who, first, seed]
chosen []
seed [the, chosen]
beginning [the]
heav'ns []
earth [the, and]
rose [how, earth, out, :, or, delight]
chaos []
sion []
hill [sion]
delight [if, hill, thee, ,, and, brook]
thee [more]
siloa ['s]
brook [siloa, fast, by]
flow'd []
fast [that, flow'd]
oracle [the, of]
god []
invoke [restore, ;, i, aid, to, ,, pursues, .]
thy []
aid [thy]
adventrous []
song [my, adventrous, ,, intends]
middle []
flight [no, middle]

In [3]:
# Creating our tokenzer function
def spacy_tokenizer(sentence):
    """This function accepts a sentence as input and processes the sentence into tokens, performing lemmatization, 
    lowercasing, removing stop words and punctuations."""
    
    # Creating our token object which is used to create documents with linguistic annotations
    mytokens = parser(sentence)
    
    # lemmatizing each token and converting each token in lower case
    # Note that spaCy uses '-PRON-' as lemma for all personal pronouns lkike me, I etc
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in STOP_WORDS and word not in punctuations]
    
    # Return preprocessed list of tokens
    return mytokens    


def create_word_counts_by_pos(raw_text, list_of_pos, word_count_dict_input = None):
    """
    takes a raw text file
    tokenizes and lemmatizes it
    limits inspection to list_of_pos types of words
    counts the individual lemmas
    returns a dictionary, keys are pos's in list_of_pos
    values are dictinaries with word counts
    """

    doc = nlp(raw_text)

    if word_count_dict_input is None: 
        word_count_dict = {}
        for part_of_speech in list_of_pos:
            word_count_dict[part_of_speech] = {}
    else:
        word_count_dict = word_count_dict_input

    for token in doc: 
        part_of_speech = token.pos_

        if part_of_speech in list_of_pos and token.is_stop == False:
            word_lemma = token.lemma_
            current_count = word_count_dict[part_of_speech].get(word_lemma, 0)
            current_count += 1
            word_count_dict[part_of_speech][word_lemma] = current_count

    return word_count_dict

# Basic function to clean the text
def clean_text(text):
    """Removing spaces and converting the text into lowercase"""
    return text.strip().lower()    

In [39]:
path = '../data/plaintext-example-files/'
os.listdir('../data/plaintext-example-files')


['adjs.txt',
 'austen.txt',
 'frankenstein.txt',
 'frost.txt',
 'genesis.txt',
 'nouns.txt',
 'plain_text_wikipedia.txt',
 'README.md',
 'samatar.txt',
 'sea_rose.txt',
 'sonnets.txt',
 'sowpods.txt',
 'verbs.txt']

In [40]:
with open(path+'genesis.txt') as infile:
    infile = infile.read()
    doc = nlp(infile)
doc

In the beginning God created the heaven and the earth. 
And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters. 
And God said, Let there be light: and there was light. 
And God saw the light, that it was good: and God divided the light from the darkness. 
And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day. 
And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters. 
And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so. 
And God called the firmament Heaven. And the evening and the morning were the second day. 
And God said, Let the waters under the heaven be gathered together unto one place, and let the dry land appear: and it was so. 
And God called the dry land Earth; and the gathering together

In [41]:
displacy.render(doc, style="ent")

In [16]:
doc[0:500]

PRIDE AND PREJUDICE

By Jane Austen



Chapter 1


It is a truth universally acknowledged, that a single man in possession
of a good fortune, must be in want of a wife.

However little known the feelings or views of such a man may be on his
first entering a neighbourhood, this truth is so well fixed in the minds
of the surrounding families, that he is considered the rightful property
of some one or other of their daughters.

"My dear Mr. Bennet," said his lady to him one day, "have you heard that
Netherfield Park is let at last?"

Mr. Bennet replied that he had not.

"But it is," returned she; "for Mrs. Long has just been here, and she
told me all about it."

Mr. Bennet made no answer.

"Do you not want to know who has taken it?" cried his wife impatiently.

"_You_ want to tell me, and I have no objection to hearing it."

This was invitation enough.

"Why, my dear, you must know, Mrs. Long says that Netherfield is taken
by a young man of large fortune from the north of England; that he

In [27]:
doc_counts = create_word_counts_by_pos(infile, ['NOUN', 'VERB','ADJ', 'ADV'], word_count_dict_input = None)
df = pd.DataFrame(doc_counts)
nouns = df['NOUN'].sort_values(ascending=False)
verbs = df['VERB'].sort_values(ascending=False)
adjectives = df['ADJ'].sort_values(ascending=False)
adverbs = df['ADV'].sort_values(ascending=False)

In [30]:
pd.DataFrame(verbs)

,VERB
let,14.0
say,11.0
see,7.0
yield,5.0
call,5.0
...,...
male,NaN
green,NaN
sixth,NaN
forth,NaN


In [31]:
list(doc.noun_chunks)

[God,
 the heaven,
 the earth,
 the earth,
 form,
 darkness,
 the face,
 the Spirit,
 God,
 the face,
 the waters,
 God,
 God,
 the light,
 it,
 God,
 the light,
 the darkness,
 God,
 the light Day,
 the darkness,
 he,
 the evening,
 the morning,
 the first day,
 God,
 a firmament,
 the midst,
 the waters,
 it,
 the waters,
 the waters,
 God,
 the firmament,
 the waters,
 the firmament,
 the waters,
 the firmament,
 it,
 God,
 the firmament Heaven,
 the evening,
 the morning,
 the second day,
 God,
 the waters,
 the heaven,
 one place,
 the dry land,
 it,
 God,
 the dry land,
 Earth,
 the gathering,
 the waters,
 God,
 it,
 God,
 the earth,
 grass,
 the herb,
 seed,
 the fruit tree,
 fruit,
 his kind,
 whose seed,
 itself,
 the earth,
 it,
 the earth,
 grass,
 seed,
 his kind,
 the tree,
 fruit,
 whose seed,
 itself,
 his kind,
 God,
 it,
 the evening,
 the morning,
 the third day,
 God,
 lights,
 the firmament,
 the heaven,
 the day,
 the night,
 them,
 signs,
 seasons,
 days,
 them,


In [43]:
for s in doc.sents:
    print(s)

In the beginning God created the heaven and the earth. 

And the earth was without form, and void; and darkness was upon the face of the deep.
And the Spirit of God moved upon the face of the waters. 

And God said, Let there be light: and there was light. 

And God saw the light, that it was good: and God divided the light from the darkness. 

And God called the light Day, and the darkness he called Night.
And the evening and the morning were the first day. 

And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters. 

And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so. 

And God called the firmament Heaven.
And the evening and the morning were the second day. 

And God said, Let the waters under the heaven be gathered together unto one place, and let the dry land appear: and it was so. 

And God called the dry land Earth; and the gathering

In [42]:
for e in doc.ents:
    print(e)

God
God
God
the light Day
Night
the evening
the first day
God
God
God
the evening
the second day
God
one
Earth
God
the evening
the third day
God
the day
the night
two
the day
the night
the day
the night
the evening
the fourth day
God
God
the evening
the fifth day
God
God
God
God
Behold
the evening
the sixth day
